In [ ]:
# Install necessary packages (for notebook environments only)
# These should be run in a cell, not a script
# !pip install -q transformers accelerate bitsandbytes huggingface_hub
# !pip install -q torchvision

from config import HUGGINGFACE_TOKEN
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    BlipProcessor,
    BlipForQuestionAnswering,
)
from huggingface_hub import HfFolder
from PIL import Image
from io import BytesIO
import requests
import torch

# Authenticate with Hugging Face Hub
# Replace with your actual token securely in production
HfFolder.save_token(HUGGINGFACE_TOKEN)

# Detect compute device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BLIP VQA processor and model for visual question answering
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(
    device
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 

2025-04-14 15:20:28.166625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744644028.646136      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744644028.769709      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
# === Load Mistral for Response Generation ===

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Define model ID for Mistral
mistral_model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Configure 4-bit quantization for efficient inference
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(mistral_model_id)

# Ensure tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load quantized Mistral model with automatic device placement
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_model_id, quantization_config=bnb_config, device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def get_blip_answer(pil_image, question):
    """
    Uses the BLIP VQA model to generate an answer to a visual question.

    Args:
        pil_image (PIL.Image.Image): The input image.
        question (str): The question to ask about the image.

    Returns:
        str: The generated answer or an error message.
    """
    try:
        # Ensure the image is in RGB format
        if pil_image.mode != "RGB":
            pil_image = pil_image.convert("RGB")

        # Preprocess image and question
        inputs = blip_processor(
            images=pil_image, text=question, return_tensors="pt"
        ).to(blip_model.device)

        # Generate answer using the model
        with torch.no_grad():
            output = blip_model.generate(**inputs, max_length=200)

        # Decode and return the generated answer
        return blip_processor.tokenizer.decode(output[0], skip_special_tokens=True)

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def get_model_answer(blip_answer, question):
    """
    Generates a formal, complete response to a user's question about a painting,
    using a language model to reframe the BLIP-provided answer.

    Args:
        blip_answer (str): The visual answer provided by the BLIP model.
        question (str): The user's question about the painting.

    Returns:
        str: A refined and formal textual response.
    """
    # Construct the prompt to guide the language model
    prompt = f"""
    You are a guide for a visually impaired person. Below is a question about a painting,
    along with an answer from another chatbot. Your task is to frame the response
    with respect to the question in a complete sentence. Be formal.

    User's Question: {question}
    Answer (from BLIP-2): {blip_answer}

    Answer:
    """

    # Tokenize input and move tensors to the model's device
    inputs = tokenizer(
        prompt, return_tensors="pt", truncation=True, padding=True, max_length=1024
    )
    inputs = {k: v.to(mistral_model.device) for k, v in inputs.items()}

    # Generate model response
    with torch.no_grad():
        outputs = mistral_model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=200,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode and clean the response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = (
        full_response.split("Answer:")[-1].strip()
        if "Answer:" in full_response
        else full_response
    )

    return answer

In [ ]:
# Install Gradio (for notebook environments; remove if running in scripts)
# !pip install gradio

import gradio as gr
from PIL import Image
import datetime
import os

# Make sure these functions are available in your script or notebook
# from your_module import get_blip_answer, get_model_answer


def process_image_and_question(image, question):
    """
    Processes an uploaded image and a user question to generate a formal answer.

    Args:
        image (PIL.Image.Image): Uploaded image.
        question (str): User's question about the image.

    Returns:
        str: Final model-generated response.
    """
    if image is None or not question:
        return "Please provide both an image and a question."

    # Save the uploaded image locally with a timestamped filename
    save_dir = "saved_images"
    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(save_dir, f"uploaded_{timestamp}.png")
    image.save(filename)
    print(f"Image saved at: {filename}")

    # Generate responses
    blip_response = get_blip_answer(image, question)
    model_response = get_model_answer(blip_response, question)

    print(f"Final Response: {model_response}")
    return model_response


# Define Gradio interface
iface = gr.Interface(
    fn=process_image_and_question,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Textbox(label="Your Question"),
    ],
    outputs="text",
    title="Image-based Q&A Interface",
    description="Upload an image, ask a question, and get an AI-generated response.",
)

# Launch the app (set share=True for public links)
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://00e4dffac7d946c1bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Image saved at: saved_images/uploaded_20250414_153614.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Final Response: The image depicts a door.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",

...

Final Response: "As a guide for the visually impaired, I can confirm that the painting depicts a door."
Image saved at: saved_images/uploaded_20250414_170402.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Final Response: As a guide for a visually impaired person, I can tell you that the painting depicts a door.
Image saved at: saved_images/uploaded_20250414_170537.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Final Response: The painting uses shades of blue as its primary color palette.
